**Load HyperModularity package**

Right now this requires activating the package. 
Later when the package is registered this will be replaced with a simple "using HyperModularity" statement.

In [42]:
using Revise
using Pkg; Pkg.activate("../.")
using HyperModularity

Activating environment at `~/GitHubRepos/Working-directory/HyperModularity/Project.toml`


**Load dataset**

Alternately, read in hypergraph stored a different way. See demo notebook on reading in hypergraphs and coverting between storage types.

In [29]:
dataset = "contact-high-school-classes"
maxsize = 5	# max hyperedge size
minsize = 2	# min hyperedge size
return_labels = true
H, L = read_hypergraph_data(dataset,maxsize,minsize,return_labels)

# In many cases it is convenient to have the hypergraph stored as an edgelist and weights vector
EdgeList, weights = HyperModularity.hyperedge_formatting(H);

(Array{Int64,1}[[21, 102, 167, 323], [28, 32, 115, 116], [33, 64, 65, 115], [49, 158, 168, 187], [58, 77, 163, 245], [144, 145, 269, 285], [17, 108, 176, 178], [145, 166, 276, 285], [207, 208, 256, 263], [4, 86, 115, 121]  …  [18, 20, 298], [139, 198, 241], [145, 269, 285], [82, 83, 127, 163, 186], [58, 77, 127, 163, 245], [144, 145, 166, 276, 285], [28, 29, 76, 85, 92], [205, 258, 265, 288, 303], [28, 41, 76, 85, 91], [145, 166, 268, 269, 276]], [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0  …  1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0])

**Simple Version**

Use the simplest version of all-or-nothing Louvain. This does not require you to set intensity function Ω. Instead, one is learned from an initial clustering. Select one of the following options for input "startclusters":

* "singletons": learn Ω from clustering where all nodes are in singletons
* "cliquelouvain": learn Ω from clustering obtained by performing clique expansion and running graph louvain
* "starlouvain": learn Ω from clustering obtained by star expansion + graph louvain.

This can be used as a first step, but it is highly recommended to alternate between learning cluster vector Z
and learning Ω intensity function, especially if using one of the simple Ω functions here.

In [26]:
start = "cliquelouvain"
gamma_res = 3.0 # can additionally toggle the resolution parameter for expansion + louvain initializer
Z = Simple_AON_Louvain(H,startclusters = start; gamma = gamma_res);

One step of all-or-nothing HyperLouvain

Louvain Iteration 1
Louvain Iteration 2
Main loop took 0.0047321319580078125 seconds
One step of all-or-nothing HyperLouvain

Louvain Iteration 1
No nodes moved clusters
Main loop took 0.00010800361633300781 seconds


**Main Version**

You can also run an all-or-nothing Louvain after just setting vectors of parameters γ and β; weights for the volume part and the cut part of the objective respectively. See paper for details, in particular the relationship with intensity funciton Ω.

Optional parameters:

* **maxits** -- maximum iterations of main step of Louvain. Default = 100
* **verbose** -- if true, print out algorithm progress. Default = true
* **clusterpenalty** -- extra penalty on number of clusters; encourages fewer clusters. Default = 0.0
* **Z0** -- warm start clustering, on which Louvain improves modularity. Default is all singletons
* **rangflag** -- if true, scan order of nodes is random. Default is false


In [43]:
β, γ = HyperModularity.Kaminski_default(H) # uses special case as defined by Kaminski et al. 2020
Z = AON_Louvain(H,β,γ; maxits = 20);

One step of all-or-nothing HyperLouvain

Louvain Iteration 1
Louvain Iteration 2
Louvain Iteration 3
Main loop took 0.013227224349975586 seconds
One step of all-or-nothing HyperLouvain

Louvain Iteration 1
Louvain Iteration 2
Main loop took 0.0002961158752441406 seconds
One step of all-or-nothing HyperLouvain

Louvain Iteration 1
No nodes moved clusters
Main loop took 9.393692016601562e-5 seconds


**To do**

Write function that converts back and forth between Omega intensities and parameter vectors β, γ.

Write function that prints out modularity objective as well as the likelihood function for these outputs, based only on β, γ.

Give example of running a coordinate ascent method where we alternate between learning Z and learning Omega.